In [2]:
camera.stop()

In [3]:
lfrobot.lfStop()

In [1]:
import ipywidgets as widgets
import traitlets
from IPython.display import display
from jetbot import Camera
import cv2
import joblib
import numpy as np
import time
import lfrobot
import numpy as np

def center_crop(image, crop_size):

    img_height, img_width = image.shape[:2]
    crop_height, crop_width = crop_size

    # Calculate the center of the image
    center_y, center_x = img_height // 2, img_width // 2

    # Calculate the cropping box
    crop_y1 = max(0, center_y - crop_height // 2)
    crop_y2 = min(img_height, center_y + crop_height // 2)
    crop_x1 = max(0, center_x - crop_width // 2)
    crop_x2 = min(img_width, center_x + crop_width // 2)

    # Return the cropped image
    return image[crop_y1:crop_y2, crop_x1:crop_x2]

# Function to convert a BGR image to JPEG format (bytes)
def bgr8_to_jpeg(value):
    _, jpeg = cv2.imencode('.jpg', value)
    return jpeg.tobytes()

# Load the pre-trained SVM model (make sure the path is correct)
svm_model = joblib.load("/workspace/jetbot/notebooks/model/model.joblib")

# Class labels
class_labels = {
    0: '55',
    1: 'Green',
    2: 'Line',
    3: 'Red',
    4: 'Sheep',
    5: 'Stop'
}

# Preprocessing function for image before prediction
def preprocess_image(image):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize the image to the same size used for training (32x32)
    crop_size = (136, 136)
    img_cropped = center_crop(img_rgb, crop_size)
    img_resized = cv2.resize(img_cropped, (32, 32))
    
    # Flatten the image
    img_flattened = img_resized.reshape(1, -1)  # Flatten the image into a single row
    
    return img_flattened


# Add a global variable to store the previous prediction
prev_prediction = None

# Function to handle robot's action based on predicted class
def handle_robot_action(predicted_class_label):
    global speed, prev_prediction
    
    # Skip the "Stop" action if the previous prediction was already "Stop"
    if predicted_class_label == 'Stop' and prev_prediction == 'Stop':
        return  # Do nothing if already in 'Stop' state
    
    if predicted_class_label == 'Stop':
        lfrobot.lfSpeed(0)  # Stop the robot
        time.sleep(3)  # Wait for 3 seconds
        speed = 0.1  # Reset speed to normal
        lfrobot.lfSpeed(speed)  # Resume normal speed
        time.sleep(1)
    
    elif predicted_class_label == '55':
        # Speed up the robot
        speed = 0.15
        lfrobot.lfSpeed(speed)
    
    elif predicted_class_label == 'Green':
        # Continue moving
        lfrobot.lfSpeed(speed)
    
    elif predicted_class_label == 'Red':
        # Stop the robot
        lfrobot.lfSpeed(0)
    
    elif predicted_class_label == 'Sheep':
        # Stop indefinitely until the sheep disappears
        lfrobot.lfSpeed(0)
    
    elif predicted_class_label == 'Line':
        # Resume movement if 'Line' is detected
        lfrobot.lfSpeed(speed)
    
    # Update the previous prediction after handling the current one
    prev_prediction = predicted_class_label

# Function to handle predicting the class from the live camera feed and act accordingly
def update_prediction(change):
    # Capture the current camera frame
    frame = change['new']
    
    # Preprocess the frame
    img_for_prediction = preprocess_image(frame)
    
    # Predict the class probabilities using the SVM model
    probabilities = svm_model.predict_proba(img_for_prediction)[0]
    
    # Get the predicted class index and label
    predicted_class_idx = np.argmax(probabilities)
    predicted_class_label = class_labels[predicted_class_idx]
    predicted_class_prob = probabilities[predicted_class_idx]  # Get the probability of the predicted class
    
    # Format the prediction probabilities as percentages
    prob_percentages = [f"{class_labels[i]}: {prob*100:.2f}%" for i, prob in enumerate(probabilities)]
    
    # Update the predicted class text widget with the class and probabilities
    predicted_label_widget.value = f'Predicted: {predicted_class_label}\n' + '\n'.join(prob_percentages)
    
    # Act on the predicted class
    if predicted_class_prob > 0.4:
        handle_robot_action(predicted_class_label)
    
    # Update the camera widget with the current frame in JPEG format
    camera_widget.value = bgr8_to_jpeg(frame)

# Initialize camera
camera = Camera()

# Create image preview widget
camera_widget = widgets.Image(format='jpeg', width=camera.width, height=camera.height)

# Create a text widget to display predicted class and probabilities
predicted_label_widget = widgets.Textarea(value='', description='Prediction', layout=widgets.Layout(width='300px', height='120px'))

# Observe the camera's value to trigger updates
camera.observe(update_prediction, names='value')

# Create a layout for displaying the camera feed and predictions
data_collection_widget = widgets.VBox([
    widgets.HBox([camera_widget]),
    predicted_label_widget
])

# Display the widgets
display(data_collection_widget)

# Initialise the robot's movement
lfrobot.lfInit()
speed = 0.1
lfrobot.lfSpeed(speed)
lfrobot.lfStart()

{'P1_REVISION': 1, 'RAM': '4096M, 2048M', 'REVISION': 'Unknown', 'TYPE': 'Jetson Nano', 'MANUFACTURER': 'NVIDIA', 'PROCESSOR': 'ARM A57'}
Initilising Line Following Jetbot
speed0
speed0.1
stop0
0.1
speed0
0
speed0
speed0
speed0
speed0
speed0
speed0
speed0
speed0
speed0
speed0.1
0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0.1
speed0